In [4]:
import pennylane as qml
import numpy as np
from pennylane import numpy as pnp
from matplotlib import pyplot as plt
from pennylane.operation import Operation, AnyWires
import os
import pandas as pd
import matplotlib.cm as cm
import math
from scipy.linalg import logm, svd

num_qubits = 5

# Initialize the device
dev = qml.device("default.qubit", wires=num_qubits)

dev2 = qml.device("default.qubit", wires=1)

In [5]:
# Construct the Hamiltonian terms
Hamiltonian_terms = []

# Interaction terms: XiX(i+1) + YiY(i+1) + ZiZ(i+1)
for i in range(num_qubits):
    Hamiltonian_terms.append(1.0 * (qml.PauliX(i) @ qml.PauliX((i+1)%num_qubits)) +
                                    (qml.PauliY(i) @ qml.PauliY((i+1)%num_qubits)) 
                                    + (qml.PauliZ(i) @ qml.PauliZ((i+1)%num_qubits)))

# Magnetic field terms: hZi
for i in range(num_qubits):
    Hamiltonian_terms.append(1.0 * qml.PauliZ(i))

# Define the Hamiltonian
Hamiltonian_operator = qml.Hamiltonian(coeffs=[1] * len(Hamiltonian_terms), observables=Hamiltonian_terms)

In [6]:
Hamiltonian = qml.Hamiltonian(coeffs=[1] * len(Hamiltonian_terms), observables=Hamiltonian_terms)

In [7]:

class fraxis_op(qml.operation.Operation):
    num_params = 1
    num_wires = qml.operation.AnyWires
    par_domain = "R"

    @staticmethod
    def compute_matrix(axis): 
        """Custom operation for free-axis rotation"""
        x, y, z = axis
        op = - 1j * qml.sum(x * qml.X(AnyWires), y * qml.Y(AnyWires), z * qml.Z(AnyWires))

        return op.matrix()

In [8]:


def entangling_layer_ladderZ(num_qubits):
    m = 0
    n = 1
    while m+1 < num_qubits:
        qml.CZ(wires=[m,m+1])
        m+=2
    
    while n+1 < num_qubits:
        qml.CZ(wires=[n,n+1])
        n+=2

@qml.qnode(dev)
def circuit(n_vectors, num_layers):
    """Parameterized quantum circuit with free-axis rotations"""
    
    for j in range(num_layers):
        for k in range(num_qubits):
            fraxis_op(n_vectors[k + num_qubits * j], wires = k)
    
        entangling_layer_ladderZ(num_qubits)

    return qml.expval(Hamiltonian)

@qml.qnode(dev)
def circuit_state(n_vectors, num_layers, d, gate_type):
    """Parameterized quantum circuit with free-axis rotations"""
    ind = 0
    for j in range(num_layers):
        for k in range(num_qubits):
            if ind == d:
                if gate_type == "X":
                    fraxis_op([1,0,0], wires=k)

                elif gate_type == "Y":
                    fraxis_op([0,1,0], wires=k)

                elif gate_type == "Z":
                    fraxis_op([0,0,1], wires=k)

                elif gate_type == "XY":
                    fraxis_op([1/np.sqrt(2), 1/np.sqrt(2), 0], wires=k)
   
                elif gate_type == "XZ":
                    fraxis_op([1/np.sqrt(2), 0, 1/np.sqrt(2)], wires=k)

                elif gate_type == "YZ":
                    fraxis_op([0, 1/np.sqrt(2), 1/np.sqrt(2)], wires=k)

            else:
                fraxis_op(n_vectors[k + num_qubits * j], wires = k)

            ind += 1
    
        entangling_layer_ladderZ(num_qubits)

    
    return qml.expval(Hamiltonian)


In [9]:

def compute_Rd_matrix(n_vectors, num_layers, d):
    """Compute the Rd matrix for a specific gate d"""

    rx = circuit_state(n_vectors, num_layers, d, gate_type="X")
    ry = circuit_state(n_vectors, num_layers, d, gate_type="Y")
    rz = circuit_state(n_vectors, num_layers, d, gate_type="Z")
    rxy = circuit_state(n_vectors, num_layers, d, gate_type="XY")
    rxz = circuit_state(n_vectors, num_layers, d, gate_type="XZ")
    ryz = circuit_state(n_vectors, num_layers, d, gate_type="YZ")
    
    Rd=[[2*rx,        2*rxy-rx-ry, 2*rxz-rx-rz],
        [2*rxy-rx-ry,        2*ry, 2*ryz-ry-rz],
        [2*rxz-rx-rz, 2*ryz-ry-rz,        2*rz]]

    return Rd



In [10]:
Id = np.matrix([[1,0],
               [0,1]])

X = np.matrix([[0,1],
               [1,0]])

Y = np.matrix([[0,-1j],
               [1j,0]])

Z = np.matrix([[1,0],
               [0,-1]])

In [ ]:
def axis_to_unitary(vec):
    x,y,z = vec
    unitary = -1j * (x*X + y*Y + z*Z)
    return unitary

def is_unitary(U):
    return np.allclose(U.conj().T @ U, np.eye(U.shape[0]))


def bloch_dist(U, V):
    inner_product = np.abs(np.trace(U.conj().T @ V))
    dist = np.sqrt(2 - inner_product)
    return dist


In [ ]:


def fraxis_optimization(n_vectors, num_layers, iters, freeze_threshold, freeze_iters):
    """Implement the Fraxis algorithm"""
    num_gates = len(n_vectors)
    vals = []
    dists = []

    freeze_counters = np.zeros(len(n_vectors))    
    
    gate_opts_tresh = num_qubits * num_layers * iters 
    gate_opts = 0

    while True:
        
        if gate_opts > gate_opts_tresh:
            break
        
        for d in range(num_gates):

            if gate_opts > gate_opts_tresh:
                break
            
            if freeze_counters[d] > 0:
                freeze_counters[d] = freeze_counters[d] - 1
                continue

            prev_axis = np.array(n_vectors[d].copy())

            current_val = circuit(n_vectors, num_layers)
            vals.append(current_val)
            Rd = compute_Rd_matrix(n_vectors, num_layers, d)        

            eigVal, eigVec = np.linalg.eig(Rd)
            eigVec = np.transpose(eigVec)

            sid = np.argmin(eigVal)

            expected_val = np.amin(eigVal)*0.5

            new_vec = [eigVec[sid][0], eigVec[sid][1], eigVec[sid][2]] 
            
            if expected_val < current_val:
                n_vectors[d] = new_vec


            current_axis = np.array(n_vectors[d].copy())

            prev_unitary = axis_to_unitary(prev_axis)
            current_unitary = axis_to_unitary(current_axis)

            
            bloch_dist_normalized = bloch_dist(prev_unitary, current_unitary) / np.sqrt(2.0)

            if (bloch_dist_normalized < freeze_threshold):
                freeze_counters[d] = freeze_iters[d]
                freeze_iters[d] += 1
                
            gate_opts += 1

        

    return n_vectors, vals, dists, freeze_iters



In [14]:
def sample_axis():
    theta = np.random.uniform(0, np.pi)
    phi = np.random.uniform(-np.pi, np.pi)

    axis = [np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)]

    return axis

In [ ]:
# Initialize parameters and run optimization
layers = [3,5]
iters = 50
trials = 20


bloch_vals = [0.01, 0.005, 0.001]

for bloch_val in bloch_vals:

    freeze_threshold = bloch_val

    for num_layers in layers:
        for trial in range(trials):
            print("trials", trial+1)
            num_gates = num_qubits * num_layers

            freeze_iters = np.ones(num_gates)

            init_n_vectors = [sample_axis() for _ in range(num_gates)]
            init_vectors = [n / np.linalg.norm(n) for n in init_n_vectors]  # Normalize initial vectors
            
            optimal_n_vectors, opt_vals, dists, freeze_iters = fraxis_optimization(init_vectors, num_layers, iters, freeze_threshold,freeze_iters)
            
            file2 = f"BlochDist_normalized_1DHeisenberg_{num_qubits}Q_Fraxis_GateFreeze_Val{bloch_val}_FreezeIterInc_{iters}cycles_{num_layers}layers_{trials}trials_A.xlsx"   
            
            if not os.path.exists(file2):
                df2 = pd.DataFrame()
                df2.to_excel(file2)

            df2 = pd.read_excel(file2)

            if len(df2.columns) < trials:
                
                df2[f"col{len(df2.columns)}"] = pd.Series(opt_vals)
                df2.to_excel(file2,index = False)
            else:
                break
